In [1]:
# import QUANTAXIS as QA

import sys
sys.path.append('..')
import json
import pandas as pd
import polars as pl
from QHData.query import Query

No module named 'wtpy'


In [2]:

# 获取全市场主力连续合约
q = Query()
futures_list = q.fetch_future_list().select(pl.col('code')).to_series().to_list()

#  获取分钟数据
future_data = q.fetch_future_data(codes=futures_list,start_date='2023-01-01',end_date='2025-03-01',frequency='5min').sort(['datetime','code'])



In [3]:
future_data

datetime,code,open,high,low,close,volume
datetime[μs],str,f64,f64,f64,f64,i64
2022-03-09 13:35:00,"""BBL8""",410.6,410.6,410.6,410.6,0
2022-03-09 13:40:00,"""BBL8""",410.6,410.6,410.6,410.6,0
2022-03-09 13:45:00,"""BBL8""",410.6,410.6,410.6,410.6,0
2022-03-09 13:50:00,"""BBL8""",410.6,410.6,410.6,410.6,0
2022-03-09 13:55:00,"""BBL8""",410.6,410.6,410.6,410.6,0
…,…,…,…,…,…,…
2025-02-28 15:10:00,"""TSL8""",102.6,102.61,102.58,102.59,1523
2025-02-28 15:15:00,"""TFL8""",105.82,105.83,105.81,105.83,670
2025-02-28 15:15:00,"""TL8""",108.22,108.25,108.21,108.25,1946


In [4]:
import polars as pl
import duckdb

def process_future_data(df: pl.DataFrame) -> pl.DataFrame:
    """使用Polars处理期货数据

    Args:
        df: 输入的期货数据DataFrame

    Returns:
        处理后的DataFrame
    """
    # 确保输入是Polars DataFrame
    if not isinstance(df, pl.DataFrame):
        df = pl.from_pandas(df.reset_index())

    # 处理code列，移除'L8'后缀
    df = (df.with_columns(
        [   pl.col('code').str.replace('L8', '').alias('code'),
            pl.col('datetime').cast(str).str.split(' ').list.get(0).str.to_date('%Y-%m-%d').dt.strftime('%Y/%m/%d').alias('date'),
            pl.col('datetime').cast(str).str.split(' ').list.get(1).str.split('.').list.get(0).str.to_time('%H:%M:%S').alias('time')
        ])
        .select(
            [
                'date',
                'time',
                'code',
                pl.col(['open', 'high', 'low', 'close']).round(2),
                pl.col('volume').round(2)
            ])
        .sort(['code', 'date', 'time'])
    )
    
    return df

processed = process_future_data(df=future_data)
duckdb.sql('select * from processed').show()

┌────────────┬──────────┬─────────┬────────┬────────┬────────┬────────┬────────┐
│    date    │   time   │  code   │  open  │  high  │  low   │ close  │ volume │
│  varchar   │   time   │ varchar │ double │ double │ double │ double │ int64  │
├────────────┼──────────┼─────────┼────────┼────────┼────────┼────────┼────────┤
│ 2023/05/31 │ 10:40:00 │ A       │ 4984.0 │ 4987.0 │ 4983.0 │ 4984.0 │    831 │
│ 2023/05/31 │ 10:45:00 │ A       │ 4983.0 │ 4990.0 │ 4983.0 │ 4983.0 │    732 │
│ 2023/05/31 │ 10:50:00 │ A       │ 4984.0 │ 4987.0 │ 4983.0 │ 4985.0 │    593 │
│ 2023/05/31 │ 10:55:00 │ A       │ 4986.0 │ 4991.0 │ 4984.0 │ 4984.0 │    641 │
│ 2023/05/31 │ 11:00:00 │ A       │ 4984.0 │ 4986.0 │ 4980.0 │ 4982.0 │    965 │
│ 2023/05/31 │ 11:05:00 │ A       │ 4981.0 │ 4986.0 │ 4981.0 │ 4983.0 │    689 │
│ 2023/05/31 │ 11:10:00 │ A       │ 4983.0 │ 4984.0 │ 4979.0 │ 4981.0 │    785 │
│ 2023/05/31 │ 11:15:00 │ A       │ 4981.0 │ 4982.0 │ 4979.0 │ 4980.0 │    396 │
│ 2023/05/31 │ 11:20:00 │ A 

In [5]:
import os

# 生层hot合约

with open('/Users/chiang/QUANTLAB/commodities.json','r',encoding='utf-8') as f:
    data = json.load(f)

hots = ['.'.join([k,m,'HOT']) for k,v in data.items() for m in v.keys()]

# 定义文件夹路径
folder_path = 'storage/csv'

# 检查文件夹是否存在
if not os.path.exists(folder_path):
    # 如果文件夹不存在，则创建文件夹
    os.makedirs(folder_path)
    print(f"文件夹 '{folder_path}' 已创建。")
else:
    print(f"文件夹 '{folder_path}' 已存在。")


for name,group in processed.group_by('code'):

    for m in hots:
        if name[0] in [x.upper() for x in m.split('.')]:

            file_name = f'{m}_m5.csv'

            df = group.drop('code')
            df.write_csv('/'.join([folder_path,file_name]))
            print(file_name)
            

文件夹 'storage/csv' 已存在。
DCE.b.HOT_m5.csv
CZCE.AP.HOT_m5.csv
CZCE.SR.HOT_m5.csv
DCE.i.HOT_m5.csv
SHFE.rb.HOT_m5.csv
DCE.m.HOT_m5.csv
CZCE.SF.HOT_m5.csv
CFFEX.TL.HOT_m5.csv
SHFE.cu.HOT_m5.csv
DCE.rr.HOT_m5.csv
DCE.p.HOT_m5.csv
CZCE.CJ.HOT_m5.csv
CZCE.SM.HOT_m5.csv
SHFE.fu.HOT_m5.csv
DCE.l.HOT_m5.csv
SHFE.wr.HOT_m5.csv
DCE.y.HOT_m5.csv
CZCE.PX.HOT_m5.csv
DCE.cs.HOT_m5.csv
CFFEX.TF.HOT_m5.csv
CZCE.OI.HOT_m5.csv
INE.bc.HOT_m5.csv
CZCE.CY.HOT_m5.csv
CZCE.SA.HOT_m5.csv
CFFEX.IM.HOT_m5.csv
DCE.bb.HOT_m5.csv
DCE.a.HOT_m5.csv
SHFE.br.HOT_m5.csv
CZCE.TA.HOT_m5.csv
CZCE.PF.HOT_m5.csv
DCE.pp.HOT_m5.csv
DCE.fb.HOT_m5.csv
DCE.eg.HOT_m5.csv
DCE.lh.HOT_m5.csv
DCE.j.HOT_m5.csv
SHFE.sn.HOT_m5.csv
SHFE.ss.HOT_m5.csv
SHFE.sp.HOT_m5.csv
INE.sc.HOT_m5.csv
INE.ec.HOT_m5.csv
DCE.c.HOT_m5.csv
SHFE.ru.HOT_m5.csv
CZCE.RS.HOT_m5.csv
DCE.pg.HOT_m5.csv
CFFEX.IC.HOT_m5.csv
SHFE.zn.HOT_m5.csv
CZCE.RM.HOT_m5.csv
CZCE.UR.HOT_m5.csv
SHFE.pb.HOT_m5.csv
SHFE.bu.HOT_m5.csv
SHFE.ag.HOT_m5.csv
DCE.v.HOT_m5.csv
DCE.eb.HOT_m5.cs

In [17]:
import pandas as pd 
import random
import numpy as np



dt_randint = {'rb':random.sample(range(1, 1000), 100),
              'ru':random.sample(range(1, 1000), 100),
              'cu':random.sample(range(1, 1000), 100)
              }

df = pd.DataFrame(dt_randint)


In [23]:
(np.log(df/df.shift(1)).dropna().std()/np.sqrt(252))

rb    0.092437
ru    0.089707
cu    0.062572
dtype: float64